In [3]:
# Đọc file predict.txt cũ (chỉ có label), thêm idx vào đầu mỗi dòng
with open("cwe20/predict_adv_cwe20.txt", "r") as f:
    labels = f.readlines()

# Nếu file cũ không có header
with open("cwe20/predict_adv_cwe20.txt", "w") as f:
    f.write("idx\tlabel\n")  # Thêm header
    for idx, label in enumerate(labels):
        label = label.strip()
        if label:  # Bỏ qua dòng trống
            f.write(f"{idx}\t{label}\n")


In [2]:
import os
import pandas as pd
import json
import random
import numpy as np
from fga_selection import *

# ========== Tham số ==========
K = 3
max_generation = 30
alpha = 2
penalty = 0.01
decay_rate = 2
insert_position = 15
adv_file_path = "cwe_filtered/CWE_20.csv"
pre_result_path = "cwe20/predict_linevul_cwe20.txt"
output_path = "cwe20/adversarial_output_cwe20.csv"
df = pd.read_csv(adv_file_path)

# ========== Dữ liệu ban đầu ==========
ad_content = pd.read_csv("preserved_pool_attack.csv")["NoiDung"].dropna().astype(str).tolist()
label_list = df["idx"].tolist()
true_labels = df["target"].tolist()

# ========== Đọc kết quả dự đoán ==========
with open(pre_result_path, "r") as f:
    pre_result = {int(line.split("\t")[0]): int(line.split("\t")[1].strip()) for line in f if line.strip()}

# ========== Lọc mẫu bị đánh nhầm ==========
vul_idx = get_vul_idx(label_list, pre_result, true_labels)
vul_codes = get_vul_codes(df.to_dict("records"), vul_idx)

print(f"🔍 Có {len(vul_codes)} mã dễ bị tấn công được chọn để sinh mã độc.")

# ========== Fuzzy clustering ==========
centroids = centriod_init(K, min_distance=0.1)

# ========== Tạo population ban đầu ==========
pop_size = 10
pop_dict = {}
for _ in range(pop_size):
    snippet = random.choice(ad_content)
    score = get_fitness_score(pre_result_path, adv_file_path, snippet_len=len(snippet), penalty=penalty)
    pop_dict[snippet] = score

# ========== Tiến hóa ==========
for gen in range(max_generation):
    print(f"⚙️ Thế hệ {gen+1}/{max_generation}")
    new_offspring = []
    offspring_scores = []

    for _ in range(pop_size):
        parent_snippet = select(pop_dict, np.mean(list(pop_dict.values())), centroids, decay_rate)
        mutated = parent_snippet + " " + random.choice(ad_content)
        mutated_score = get_fitness_score(pre_result_path, adv_file_path, snippet_len=len(mutated), penalty=penalty)
        new_offspring.append(mutated)
        offspring_scores.append(mutated_score)

    pop_dict = update_global_pop(new_offspring, pop_dict, offspring_scores)

# ========== Lấy snippet tốt nhất ==========
best_snippet = max(pop_dict, key=pop_dict.get)
print(f"🏆 Best adversarial snippet:\n{best_snippet}")

# ========== Chèn mã độc vào processed_func ==========
vul_rows = df[df["idx"].isin(vul_idx)]
vul_codes = dict(zip(vul_rows["idx"], vul_rows["processed_func"]))
adv_codes, _ = add_adver_sample_2_ast(vul_codes, insert_position, best_snippet.split())

# ========== Ghi lại DataFrame gốc với mã độc đã thay thế ==========
df_adv = df.copy()
for i, idx in enumerate(vul_rows.index):
    df_adv.at[idx, "processed_func"] = adv_codes[i]

df_adv.to_csv(output_path, index=False)
print(f"File mã độc đã ghi vào: {output_path}")


C:\Users\Admin\AppData\Local\Temp\ipykernel_9400\4174520569.py:18: DtypeWarning: Columns (27,28,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(adv_file_path)


🔍 Có 949 mã dễ bị tấn công được chọn để sinh mã độc.


d:\Nam3_ANTN\ML\DA\Coding\fga_selection.py:39: DtypeWarning: Columns (27,28,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(adv_file_path)
d:\Nam3_ANTN\ML\DA\Coding\fga_selection.py:39: DtypeWarning: Columns (27,28,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(adv_file_path)
d:\Nam3_ANTN\ML\DA\Coding\fga_selection.py:39: DtypeWarning: Columns (27,28,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(adv_file_path)
d:\Nam3_ANTN\ML\DA\Coding\fga_selection.py:39: DtypeWarning: Columns (27,28,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(adv_file_path)
d:\Nam3_ANTN\ML\DA\Coding\fga_selection.py:39: DtypeWarning: Columns (27,28,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(adv_file_path)
d:\Nam3_ANTN\ML\DA\Coding\fga_selection.py:39

⚙️ Thế hệ 1/30


d:\Nam3_ANTN\ML\DA\Coding\fga_selection.py:39: DtypeWarning: Columns (27,28,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(adv_file_path)
d:\Nam3_ANTN\ML\DA\Coding\fga_selection.py:39: DtypeWarning: Columns (27,28,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(adv_file_path)
d:\Nam3_ANTN\ML\DA\Coding\fga_selection.py:39: DtypeWarning: Columns (27,28,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(adv_file_path)
d:\Nam3_ANTN\ML\DA\Coding\fga_selection.py:39: DtypeWarning: Columns (27,28,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(adv_file_path)
d:\Nam3_ANTN\ML\DA\Coding\fga_selection.py:39: DtypeWarning: Columns (27,28,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(adv_file_path)
d:\Nam3_ANTN\ML\DA\Coding\fga_selection.py:39

...

⚙️ Thế hệ 30/30


d:\Nam3_ANTN\ML\DA\Coding\fga_selection.py:39: DtypeWarning: Columns (27,28,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(adv_file_path)
d:\Nam3_ANTN\ML\DA\Coding\fga_selection.py:39: DtypeWarning: Columns (27,28,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(adv_file_path)
d:\Nam3_ANTN\ML\DA\Coding\fga_selection.py:39: DtypeWarning: Columns (27,28,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(adv_file_path)
d:\Nam3_ANTN\ML\DA\Coding\fga_selection.py:39: DtypeWarning: Columns (27,28,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(adv_file_path)
d:\Nam3_ANTN\ML\DA\Coding\fga_selection.py:39: DtypeWarning: Columns (27,28,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(adv_file_path)
d:\Nam3_ANTN\ML\DA\Coding\fga_selection.py:39

🏆 Best adversarial snippet:

	for (int i = 0
		break
	}




100%|██████████| 949/949 [00:00<00:00, 67323.96it/s]


File mã độc đã ghi vào: cwe20/adversarial_output_cwe20.csv


In [4]:
import pandas as pd
from sklearn.metrics import accuracy_score

# === Load original prediction và adversarial prediction ===
def load_preds(path):
    with open(path) as f:
        return {int(l.split('\t')[0]): int(l.split('\t')[1]) for l in f if l.strip()}

orig_pred = load_preds("cwe20/predict_linevul_cwe20.txt")
adv_pred = load_preds("cwe20/predict_adv_cwe20.txt")

# === Load ground-truth labels từ CSV ===
df = pd.read_csv("cwe20/adversarial_output_cwe20.csv") 
labels = dict(zip(df["idx"], df["target"]))

# === Xác định mẫu vulnerable đúng bị phát hiện ban đầu ===
vul_correct_ids = [idx for idx in labels if labels[idx] == 1 and orig_pred.get(idx, 0) == 1]

# === Sau tấn công: bị đánh nhầm là benign ===
successful_attack_ids = [idx for idx in vul_correct_ids if adv_pred.get(idx, 1) == 0]

# === Tính ASR (Attack Success Rate) ===
ASR = len(successful_attack_ids) / len(vul_correct_ids) if vul_correct_ids else 0
print(f"ASR = {ASR:.3%} ({len(successful_attack_ids)}/{len(vul_correct_ids)})")


ASR = 85.985% (816/949)


C:\Users\Admin\AppData\Local\Temp\ipykernel_9400\2148970734.py:13: DtypeWarning: Columns (27,28,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("cwe20/adversarial_output_cwe20.csv")


In [7]:
import pandas as pd

# Đọc file CSV
df = pd.read_csv("processed_data.csv")  # thay bằng tên file thật

# Lọc các hàng có "OpenSSL" trong cột "Vulnerability Classification"
filtered_df = df[df["Vulnerability Classification"] == "DoS"]

# Hiển thị kết quả
print(filtered_df)

# (Tuỳ chọn) Lưu ra file mới
filtered_df.to_csv("filtered_dos.csv", index=False)

C:\Users\Admin\AppData\Local\Temp\ipykernel_9400\951952044.py:4: DtypeWarning: Columns (1,20,22,23,27,28,29,37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("processed_data.csv")  # thay bằng tên file thật


         index Access Gained Attack Origin Authentication Required  \
76          76           NaN        Remote            Not required   
77          77           NaN        Remote            Not required   
78          78           NaN        Remote            Not required   
79          79           NaN        Remote            Not required   
80          80           NaN        Remote            Not required   
...        ...           ...           ...                     ...   
178625  178625           NaN        Remote            Not required   
178626  178626           NaN        Remote            Not required   
178627  178627           NaN        Remote            Not required   
178655  178655           NaN        Remote            Not required   
178656  178656           NaN        Remote            Not required   

       Availability          CVE ID  \
76          Partial   CVE-2013-6449   
77          Partial   CVE-2013-6449   
78          Partial   CVE-2013-6449   
79 

In [ ]:
import pandas as pd

def count_target_for_selected_cwes(file_path, selected_cwes):
    # Đọc file CSV
    df = pd.read_csv(file_path)

    # Chỉ giữ lại các dòng có CWE ID nằm trong danh sách chỉ định
    df_filtered = df[df["Vulnerability Classification"].isin(selected_cwes)]

    # Nhóm theo 'CWE ID' và 'target', đếm số lượng
    result = df_filtered.groupby(['Vulnerability Classification', 'target']).size().unstack(fill_value=0)

    # Đảm bảo đủ 2 cột target
    if 0 not in result.columns:
        result[0] = 0
    if 1 not in result.columns:
        result[1] = 0

    result = result[[0, 1]]  # sắp xếp cột theo thứ tự target=0, target=1
    result.columns = ['target_0_count', 'target_1_count']

    # In kết quả
    for cwe_id, row in result.iterrows():
        print(f"CWE ID {cwe_id}: target=0 → {row['target_0_count']} mẫu, target=1 → {row['target_1_count']} mẫu")

# Danh sách các CWE cần kiểm tra        
selected_cwes = ['DoS']

# Gọi hàm
count_target_for_selected_cwes("processed_data.csv" , selected_cwes)
